In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip ./glove.6B.zip

--2022-07-10 10:04:43--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-07-10 10:04:43--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-07-10 10:04:44--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
#you could find the dataset in kaggle
latent_dim=25
emb_dim=50
max_vocabs=3000
df=pd.read_csv("/content/robert_frost_collection.csv")
data=df["Content"]
data=data.dropna()

text=[]
for line in data:
  
  temp="<sos> "+line.lower()+" <eos>"
  text.append(temp)
print(text)

['<sos> whose woods these are i think i know.   \nhis house is in the village though;   \nhe will not see me stopping here   \nto watch his woods fill up with snow.   \nmy little horse must think it queer   \nto stop without a farmhouse near   \nbetween the woods and frozen lake   \nthe darkest evening of the year.   \nhe gives his harness bells a shake   \nto ask if there is some mistake.   \nthe only other sound’s the sweep   \nof easy wind and downy flake.   \nthe woods are lovely, dark and deep,   \nbut i have promises to keep,   \nand miles to go before i sleep,   \nand miles to go before i sleep.\n <eos>', '<sos> some say the world will end in fire,\nsome say in ice.\nfrom what i’ve tasted of desire\ni hold with those who favor fire.\nbut if it had to perish twice,\ni think i know enough of hate\nto say that for destruction ice\nis also great\nand would suffice.\n <eos>', '<sos> before man came to blow it right\nthe wind once blew itself untaught,\nand did its loudest day and nig

In [ ]:
tokenizer=keras.preprocessing.text.Tokenizer(num_words=max_vocabs,filters="   ")
tokenizer.fit_on_texts(text)
text=tokenizer.texts_to_sequences(text)
word2index=tokenizer.word_index
max_sequance_len=max(len(s) for s in text)
min_sequance=min(len(s) for s in text)

In [ ]:
input_text=[]
target_text=[]
for line in text:
  for i in range(0,max_sequance_len):
    input_text.append(line[i:(min_sequance-5)+i])
    temp=line[(min_sequance-5)+i:(min_sequance-5)+i+1]
    target_text.append(temp)
    if temp==word2index["<eos>"]:
      break


In [ ]:
input_text=input_text[:10000]
target_text=target_text[:10000]
max_sequance_len=max(len(s) for s in input_text)
input_text=keras.preprocessing.sequence.pad_sequences(input_text,maxlen=max_sequance_len,padding="post")
target_text=keras.preprocessing.sequence.pad_sequences(target_text,maxlen=max_sequance_len,padding="post")

In [ ]:
# from tensorflow.keras.utils import to_categorical
# target=to_categorical(target_text)
num_word=min(len(word2index)+1,max_vocabs)
target=np.zeros((len(target_text),max_sequance_len,num_word))
for i in range(len(target_text)):
  for j in range(len(target_text[i])):
    if (input_text[i][j]-1)>-1:
     target[i,j,(input_text[i][j]-1)]=1

In [ ]:
word2vec={}
f=open("./glove.6B."+str(emb_dim)+"d.txt")    
for line in f:
        temp=line.split()
        word=temp[0]
        vec=np.asarray(temp[1:],dtype="float32")
        word2vec[word]=vec

In [ ]:
embedding_matrix=np.zeros((num_word,emb_dim))
for word,i in word2index.items():
  if i<max_vocabs:
    embedding_vec=word2vec.get(word)
    if  embedding_vec is not None:
        embedding_matrix[i]=embedding_vec

In [ ]:
embedding_layer=keras.layers.Embedding(num_word,emb_dim,weights=[embedding_matrix]
                                        ,input_length=max_sequance_len)

In [ ]:
input_=keras.layers.Input(shape=(max_sequance_len,))
initial_h=keras.layers.Input(shape=(latent_dim,))
initial_c=keras.layers.Input(shape=(latent_dim,))
x=embedding_layer(input_)
lstm=keras.layers.LSTM(latent_dim,return_sequences=True, return_state=True)
x,_,_=lstm(x,initial_state=[initial_h,initial_c])
dense=keras.layers.Dense(num_word,activation="softmax")
y=dense(x)
model=keras.Model([input_,initial_h,initial_c],y)

In [ ]:
model.compile(optimizer="adam",loss=keras.losses.CategoricalCrossentropy(),metrics=["accuracy"])

In [ ]:
z=np.zeros((len(input_text),latent_dim))
# print("input text shape: "+str(len(word2index))," output shape: "+str(target.shape)," z shape: "+str(z.shape))
model.fit([input_text,z,z]
          ,target,batch_size=32,epochs=10,validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 8s 10ms/step - loss: 0.3569 - accuracy: 0.9369 - val_loss: 0.7929 - val_accuracy: 0.9020
Epoch 2/10
250/250 [==============================] - 2s 7ms/step - loss: 0.3237 - accuracy: 0.9519 - val_loss: 0.7936 - val_accuracy: 0.9020
Epoch 3/10
250/250 [==============================] - 2s 7ms/step - loss: 0.3192 - accuracy: 0.9519 - val_loss: 0.7929 - val_accuracy: 0.9020
Epoch 4/10
250/250 [==============================] - 2s 7ms/step - loss: 0.3165 - accuracy: 0.9519 - val_loss: 0.7936 - val_accuracy: 0.9020
Epoch 5/10
250/250 [==============================] - 2s 7ms/step - loss: 0.3145 - accuracy: 0.9519 - val_loss: 0.7942 - val_accuracy: 0.9020
Epoch 6/10
250/250 [==============================] - 2s 7ms/step - loss: 0.3129 - accuracy: 0.9519 - val_loss: 0.7950 - val_accuracy: 0.9020
Epoch 7/10
250/250 [==============================] - 2s 7ms/step - loss: 0.3116 - accuracy: 0.9519 - val_loss: 0.7959 - val_accuracy: 0.9020
Epoch

In [ ]:
input2_=keras.layers.Input(shape=(1,))
x=embedding_layer(input2_)
x,h,c=lstm(x,initial_state=[initial_h,initial_c])
y2=dense(x)
model=keras.Model([input2_,initial_h,initial_c],[y2,h,c])

In [ ]:
gen_sen=""
h=np.zeros((1,latent_dim))
c=np.zeros((1,latent_dim))
_input=np.asarray([[word2index["<sos>"]]])
size=max_sequance_len
for i in range(size):
  _input,h,c=model.predict([_input,h,c])
  prop=_input[0,0]
  prop[0]=0
  prop/=prop.sum()
  idx=np.random.choice(len(prop),p=prop)
  word=tokenizer.index_word[idx]  
  _input=np.asarray([[idx]])
  if size-1==i:
    print("done before finding eos")
    print(gen_sen)
  if word=="<eos>":
    
    print(gen_sen)
    break
  else:
    if gen_sen=="":
      gen_sen+=word
    else:
      gen_sen+=" "+word        


done before finding eos
weary, has rough is;
the 
baptiste bells
